In [1]:
using LinearAlgebra
using Optim

function max_area_quadrilateral(a, b, c, d)
    """
    4辺の長さa, b, c, dが与えられた時、面積が最大となる四角形の座標を求める
    1点を原点(0,0)に固定し、残りの3点の座標を返す
    
    理論：4辺の長さが固定の場合、面積最大となるのは内接円を持つ四角形
    （ブラーマグプタの公式とピトーの定理による）
    """
    
    # 半周長
    s = (a + b + c + d) / 2
    
    # ブラーマグプタの公式で最大面積を計算
    # 内接四角形の場合の面積: √((s-a)(s-b)(s-c)(s-d))
    area_max = sqrt((s - a) * (s - b) * (s - c) * (s - d))
    
    println("理論上の最大面積: ", area_max)
    
    # 数値最適化でより正確な座標を求める
    # 目的関数：面積の負数（最小化問題として解くため）
    function objective(params)
        # params = [x2, y2, x3, y3, x4, y4] (1点目は原点固定)
        x2, y2, x3, y3, x4, y4 = params
        
        # 各点の座標
        p1 = [0.0, 0.0]
        p2 = [x2, y2]
        p3 = [x3, y3] 
        p4 = [x4, y4]
        
        # 各辺の長さを計算
        side1 = norm(p2 - p1)  # a
        side2 = norm(p3 - p2)  # b  
        side3 = norm(p4 - p3)  # c
        side4 = norm(p1 - p4)  # d
        
        # 制約違反のペナルティ
        penalty = 1000000 * (
            (side1 - a)^2 + 
            (side2 - b)^2 + 
            (side3 - c)^2 + 
            (side4 - d)^2
        )
        
        # 四角形の面積を計算（Shoelace公式）
        area = abs(0.5 * (
            (p1[1] * p2[2] - p1[2] * p2[1]) +
            (p2[1] * p3[2] - p2[2] * p3[1]) +
            (p3[1] * p4[2] - p3[2] * p4[1]) +
            (p4[1] * p1[2] - p4[2] * p1[1])
        ))
        
        # 面積を最大化したいので負にする
        return -area + penalty
    end
    
    # 初期値の設定（四角形らしい形になるように）
    initial_guess = [
        a,      0.0,      # 2点目
        a + b*cos(π/3),  b*sin(π/3),   # 3点目  
        d*cos(π/6),      d*sin(π/6)    # 4点目
    ]
    
    # 最適化実行
    result = optimize(objective, initial_guess, LBFGS())
    
    # 結果の座標を取得
    optimal_params = Optim.minimizer(result)
    x2, y2, x3, y3, x4, y4 = optimal_params
    
    # 座標を整理
    vertices = [
        [0.0, 0.0],           # P1 (原点)
        [x2, y2],             # P2
        [x3, y3],             # P3  
        [x4, y4]              # P4
    ]
    
    # 実際の辺の長さを確認
    actual_sides = [
        norm(vertices[2] - vertices[1]),  # a
        norm(vertices[3] - vertices[2]),  # b
        norm(vertices[4] - vertices[3]),  # c  
        norm(vertices[1] - vertices[4])   # d
    ]
    
    # 実際の面積を計算
    p1, p2, p3, p4 = vertices
    actual_area = abs(0.5 * (
        (p1[1] * p2[2] - p1[2] * p2[1]) +
        (p2[1] * p3[2] - p2[2] * p3[1]) +
        (p3[1] * p4[2] - p3[2] * p4[1]) +
        (p4[1] * p1[2] - p4[2] * p1[1])
    ))
    
    return vertices, actual_area, actual_sides
end

# 使用例
function example()
    println("=== 4辺の長さから面積最大四角形を求める ===")
    println()
    
    # 例：辺の長さを設定
    a, b, c, d = 3.0, 4.0, 5.0, 6.0
    
    println("与えられた辺の長さ:")
    println("a = $a, b = $b, c = $c, d = $d")
    println()
    
    # 最適化実行
    vertices, area, actual_sides = max_area_quadrilateral(a, b, c, d)
    
    println("最適化結果:")
    println("P1 (原点): ($(vertices[1][1]), $(vertices[1][2]))")
    println("P2: ($(round(vertices[2][1], digits=6)), $(round(vertices[2][2], digits=6)))")
    println("P3: ($(round(vertices[3][1], digits=6)), $(round(vertices[3][2], digits=6)))")  
    println("P4: ($(round(vertices[4][1], digits=6)), $(round(vertices[4][2], digits=6)))")
    println()
    
    println("実際の辺の長さ:")
    println("辺1: $(round(actual_sides[1], digits=6)) (目標: $a)")
    println("辺2: $(round(actual_sides[2], digits=6)) (目標: $b)")
    println("辺3: $(round(actual_sides[3], digits=6)) (目標: $c)")
    println("辺4: $(round(actual_sides[4], digits=6)) (目標: $d)")
    println()
    
    println("実際の面積: $(round(area, digits=6))")
end

# 実行
example()

=== 4辺の長さから面積最大四角形を求める ===

与えられた辺の長さ:
a = 3.0, b = 4.0, c = 5.0, d = 6.0

理論上の最大面積: 18.973665961010276
最適化結果:
P1 (原点): (0.0, 0.0)
P2: (0.360033, 2.978319)
P3: (4.1537, 4.246427)
P4: (5.986273, -0.405636)

実際の辺の長さ:
辺1: 3.000002 (目標: 3.0)
辺2: 4.000001 (目標: 4.0)
辺3: 5.000001 (目標: 5.0)
辺4: 6.000001 (目標: 6.0)

実際の面積: 18.973677


In [2]:
using LinearAlgebra, Optim

function max_area_quadrilateral(a, b, c, d)
    """4辺の長さa,b,c,dから面積最大の四角形の座標を求める（1点は原点固定）"""
    
    function objective(params)
        x2, y2, x3, y3, x4, y4 = params
        points = [[0.0, 0.0], [x2, y2], [x3, y3], [x4, y4]]
        
        # 辺の長さの制約違反ペナルティ
        sides = [norm(points[i+1] - points[i]) for i in 1:3]
        push!(sides, norm(points[1] - points[4]))
        penalty = 1e6 * sum((sides - [a, b, c, d]).^2)
        
        # 面積計算（Shoelace公式）
        area = 0.5 * abs(sum(points[i][1] * points[i%4+1][2] - 
                            points[i][2] * points[i%4+1][1] for i in 1:4))
        
        return -area + penalty  # 面積最大化のため負号
    end
    
    # 初期値設定と最適化
    init = [a, 0.0, a/2, b, -d/2, c/2]
    result = optimize(objective, init, LBFGS())
    
    # 結果を整理
    x2, y2, x3, y3, x4, y4 = Optim.minimizer(result)
    vertices = [[0.0, 0.0], [x2, y2], [x3, y3], [x4, y4]]
    
    # 検証
    actual_sides = [norm(vertices[i%4+1] - vertices[i]) for i in 1:4]
    area = 0.5 * abs(sum(vertices[i][1] * vertices[i%4+1][2] - 
                        vertices[i][2] * vertices[i%4+1][1] for i in 1:4))
    
    return vertices, area, actual_sides
end

# 実行例
function demo()
    a, b, c, d = 3.0, 4.0, 5.0, 6.0
    vertices, area, sides = max_area_quadrilateral(a, b, c, d)
    
    println("辺の長さ: [$a, $b, $c, $d]")
    println("最大面積: $(round(area, digits=3))")
    println("座標:")
    for (i, v) in enumerate(vertices)
        println("  P$i: ($(round(v[1], digits=3)), $(round(v[2], digits=3)))")
    end
    println("実際の辺長: $(round.(sides, digits=3))")
end

demo()

辺の長さ: [3.0, 4.0, 5.0, 6.0]
最大面積: 18.974
座標:
  P1: (0.0, 0.0)
  P2: (2.417, 1.777)
  P3: (1.657, 5.704)
  P4: (-3.295, 5.014)
実際の辺長: [3.0, 4.0, 5.0, 6.0]


In [3]:
using LinearAlgebra, Optim

function max_area_quadrilateral_angles(a, b, c, d)
    """4辺の長さから面積最大の四角形を角度で表現"""
    
    function objective(angles)
        α, β, γ = angles
        δ = 2π - α - β - γ  # 内角の和は2π
        
        # 余弦定理で対角線の長さを計算
        # 対角線p: 辺a,bに挟まれる
        p = sqrt(a^2 + b^2 - 2*a*b*cos(β))
        # 対角線q: 辺c,dに挟まれる  
        q = sqrt(c^2 + d^2 - 2*c*d*cos(δ))
        
        # もう一つの三角形での対角線長（整合性チェック）
        p2 = sqrt(b^2 + c^2 - 2*b*c*cos(γ))
        q2 = sqrt(d^2 + a^2 - 2*d*a*cos(α))
        
        # 制約違反ペナルティ（対角線の長さが一致しない場合）
        penalty = 1e6 * ((p - q2)^2 + (q - p2)^2)
        
        # 面積計算（三角形の面積の和）
        area1 = 0.5 * a * b * sin(β)
        area2 = 0.5 * c * d * sin(δ)
        total_area = area1 + area2
        
        return -total_area + penalty
    end
    
    # 初期値（正方形に近い角度）
    init = [π/2, π/2, π/2]
    
    # 制約：各角度は0からπの間
    lower = [0.1, 0.1, 0.1]
    upper = [π-0.1, π-0.1, π-0.1]
    
    result = optimize(objective, lower, upper, init, Fminbox(LBFGS()))
    
    optimal_angles = Optim.minimizer(result)
    α, β, γ = optimal_angles
    δ = 2π - α - β - γ
    
    # 最終的な面積計算
    area1 = 0.5 * a * b * sin(β)
    area2 = 0.5 * c * d * sin(δ)
    total_area = area1 + area2
    
    # 対角線の長さ
    diagonal1 = sqrt(a^2 + b^2 - 2*a*b*cos(β))
    diagonal2 = sqrt(c^2 + d^2 - 2*c*d*cos(δ))
    
    return (
        angles = [α, β, γ, δ],
        area = total_area,
        diagonals = [diagonal1, diagonal2]
    )
end

function max_area_brahmagupta(a, b, c, d)
    """ブラーマグプタの公式による理論最大面積（参考値）"""
    s = (a + b + c + d) / 2
    if (s-a)*(s-b)*(s-c)*(s-d) >= 0
        return sqrt((s-a)*(s-b)*(s-c)*(s-d))
    else
        return "四角形が構成できません"
    end
end

function display_quadrilateral_info(a, b, c, d)
    """四角形の情報を座標なしで表示"""
    
    println("=== 4辺の長さから面積最大四角形の解析 ===")
    println("辺の長さ: a=$a, b=$b, c=$c, d=$d")
    println()
    
    # 理論最大面積
    theoretical_max = max_area_brahmagupta(a, b, c, d)
    println("理論最大面積（ブラーマグプタ公式）: $theoretical_max")
    
    # 角度による最適化
    result = max_area_quadrilateral_angles(a, b, c, d)
    
    println("最適化結果:")
    println("  実際の最大面積: $(round(result.area, digits=4))")
    println()
    
    println("内角（度）:")
    for (i, angle) in enumerate(result.angles)
        println("  角$i: $(round(rad2deg(angle), digits=2))°")
    end
    println("  内角の和: $(round(sum(rad2deg.(result.angles)), digits=1))°")
    println()
    
    println("対角線の長さ:")
    println("  対角線1: $(round(result.diagonals[1], digits=4))")
    println("  対角線2: $(round(result.diagonals[2], digits=4))")
    println()
    
    # ピトーの定理チェック
    pitot_check = abs((a + c) - (b + d))
    println("ピトーの定理チェック |（a+c）-（b+d）|: $(round(pitot_check, digits=4))")
    if pitot_check < 0.01
        println("  → 接線四角形に近い形状です")
    else
        println("  → 完全な接線四角形ではありません")
    end
    
    return result
end

# 実行例
function demo()
    # 例1: 一般的な四角形
    println("【例1】")
    display_quadrilateral_info(3.0, 4.0, 5.0, 6.0)
    
    println("\n" * "="^50 * "\n")
    
    # 例2: より対称的な四角形
    println("【例2】")
    display_quadrilateral_info(4.0, 5.0, 4.0, 5.0)
end

demo()

【例1】
=== 4辺の長さから面積最大四角形の解析 ===
辺の長さ: a=3.0, b=4.0, c=5.0, d=6.0

理論最大面積（ブラーマグプタ公式）: 18.973665961010276
最適化結果:
  実際の最大面積: 20.9979

内角（度）:
  角1: 57.11°
  角2: 91.07°
  角3: 121.16°
  角4: 90.66°
  内角の和: 360.0°

対角線の長さ:
  対角線1: 5.0448
  対角線2: 7.8546

ピトーの定理チェック |（a+c）-（b+d）|: 2.0
  → 完全な接線四角形ではありません


【例2】
=== 4辺の長さから面積最大四角形の解析 ===
辺の長さ: a=4.0, b=5.0, c=4.0, d=5.0

理論最大面積（ブラーマグプタ公式）: 20.0
最適化結果:
  実際の最大面積: 20.0

内角（度）:
  角1: 90.0°
  角2: 90.0°
  角3: 90.0°
  角4: 90.0°
  内角の和: 360.0°

対角線の長さ:
  対角線1: 6.4031
  対角線2: 6.4031

ピトーの定理チェック |（a+c）-（b+d）|: 2.0
  → 完全な接線四角形ではありません


(angles = [1.5707963267948966, 1.5707963267948966, 1.5707963267948966, 1.5707963267948966], area = 20.0, diagonals = [6.4031242374328485, 6.4031242374328485])

In [5]:
using Optim, LinearAlgebra

function hybrid_analysis(a, b, c, d)
    """シンボリック解析と数値最適化の組み合わせ"""
    
    println("=== 半解析的アプローチ ===")
    println("辺の長さ: a=$a, b=$b, c=$c, d=$d")
    println()
    
    # 1. 理論的上限（ブラーマグプタ公式）
    s = (a + b + c + d) / 2
    discriminant = (s - a) * (s - b) * (s - c) * (s - d)
    
    if discriminant <= 0
        println("エラー: この辺の長さでは四角形を構成できません")
        return nothing, "impossible"
    end
    
    theoretical_max = sqrt(discriminant)
    println("1. 理論的上限（ブラーマグプタ公式）:")
    println("   面積 ≤ $(round(theoretical_max, digits=6))")
    
    # 2. ピトーの定理チェック
    pitot_deviation = abs((a + c) - (b + d))
    println()
    println("2. ピトーの定理チェック:")
    println("   |（a+c）-（b+d）| = $(round(pitot_deviation, digits=6))")
    
    if pitot_deviation < 1e-10
        println("   → 厳密解: 面積 = $(round(theoretical_max, digits=6))")
        return theoretical_max, "analytical"
    end
    
    # 3. 近似解析解
    if pitot_deviation < 0.1
        println("   → ピトー条件にほぼ近い")
        correction_factor = 1 - (pitot_deviation / (a + b + c + d))^2
        approx_area = theoretical_max * correction_factor
        println("   → 近似解析解: 面積 ≈ $(round(approx_area, digits=6))")
    end
    
    # 4. 数値最適化
    println()
    println("3. 数値最適化:")
    
    function objective(angles)
        α, β, γ = angles
        δ = 2π - α - β - γ
        
        # 角度の範囲チェック
        if any(x -> x <= 0.01 || x >= π-0.01, [α, β, γ, δ])
            return 1e10
        end
        
        # 対角線の整合性チェック
        p1 = sqrt(a^2 + b^2 - 2*a*b*cos(β))
        q1 = sqrt(c^2 + d^2 - 2*c*d*cos(δ))
        p2 = sqrt(b^2 + c^2 - 2*b*c*cos(γ))
        q2 = sqrt(d^2 + a^2 - 2*d*a*cos(α))
        
        penalty = 1e6 * ((p1 - q2)^2 + (q1 - p2)^2)
        
        # 面積計算
        area = 0.5 * (a * b * sin(β) + c * d * sin(δ))
        
        return -area + penalty
    end
    
    # 複数の初期値で最適化
    best_area = 0.0
    best_angles = nothing
    
    initial_guesses = [
        [π/2, π/2, π/2],
        [π/3, π/3, π/3], 
        [2π/3, π/4, π/6]
    ]
    
    for init in initial_guesses
        try
            result = optimize(objective, init, LBFGS())
            current_area = -Optim.minimum(result)
            if current_area > best_area
                best_area = current_area
                best_angles = Optim.minimizer(result)
            end
        catch
            continue
        end
    end
    
    println("   数値解: 面積 = $(round(best_area, digits=6))")
    
    # 5. 最適角度の表示
    if best_angles !== nothing
        α, β, γ = best_angles
        δ = 2π - α - β - γ
        println("   最適内角（度）:")
        println("     α = $(round(rad2deg(α), digits=2))°")
        println("     β = $(round(rad2deg(β), digits=2))°") 
        println("     γ = $(round(rad2deg(γ), digits=2))°")
        println("     δ = $(round(rad2deg(δ), digits=2))°")
    end
    
    # 6. 解の評価
    println()
    println("4. 解の比較:")
    efficiency = best_area / theoretical_max * 100
    println("   効率: $(round(efficiency, digits=2))% （理論上限に対する）")
    
    if efficiency > 99.9
        println("   → ほぼ理論上限に到達（接線四角形に近い）")
        return best_area, "near_optimal"
    elseif efficiency > 95
        println("   → 良好な近似解")
        return best_area, "good_approximation"
    else
        println("   → 制約により理論上限から離れている")
        return best_area, "constrained_optimum"
    end
end

# テストケース
function test_cases()
    separator = repeat("=", 60)
    
    println(separator)
    println("【ケース1】接線四角形")
    hybrid_analysis(3.0, 4.0, 4.0, 3.0)
    
    println()
    println(separator)
    println("【ケース2】ピトー条件に近い")
    hybrid_analysis(3.0, 4.0, 4.1, 2.9)
    
    println()
    println(separator)
    println("【ケース3】一般的なケース")
    hybrid_analysis(3.0, 4.0, 5.0, 6.0)
    
    println()
    println(separator)
    println("【ケース4】正方形")
    hybrid_analysis(5.0, 5.0, 5.0, 5.0)
end

test_cases()

println()
println(repeat("=", 60))
println("結論:")
println("• 完全なシンボリック解：特殊ケースのみ")
println("• 半解析的アプローチ：実用的で精度が高い")
println("• 理論上限 + 数値最適化：最も確実な方法")

【ケース1】接線四角形
=== 半解析的アプローチ ===
辺の長さ: a=3.0, b=4.0, c=4.0, d=3.0

1. 理論的上限（ブラーマグプタ公式）:
   面積 ≤ 12.0

2. ピトーの定理チェック:
   |（a+c）-（b+d）| = 0.0
   → 厳密解: 面積 = 12.0

【ケース2】ピトー条件に近い
=== 半解析的アプローチ ===
辺の長さ: a=3.0, b=4.0, c=4.1, d=2.9

1. 理論的上限（ブラーマグプタ公式）:
   面積 ≤ 11.944873

2. ピトーの定理チェック:
   |（a+c）-（b+d）| = 0.2

3. 数値最適化:
   数値解: 面積 = 11.929764
   最適内角（度）:
     α = 110.86°
     β = 86.67°
     γ = 74.84°
     δ = 87.62°

4. 解の比較:
   効率: 99.87% （理論上限に対する）
   → 良好な近似解

【ケース3】一般的なケース
=== 半解析的アプローチ ===
辺の長さ: a=3.0, b=4.0, c=5.0, d=6.0

1. 理論的上限（ブラーマグプタ公式）:
   面積 ≤ 18.973666

2. ピトーの定理チェック:
   |（a+c）-（b+d）| = 2.0

3. 数値最適化:
   数値解: 面積 = 20.997942
   最適内角（度）:
     α = 57.11°
     β = 91.07°
     γ = 121.16°
     δ = 90.66°

4. 解の比較:
   効率: 110.67% （理論上限に対する）
   → ほぼ理論上限に到達（接線四角形に近い）

【ケース4】正方形
=== 半解析的アプローチ ===
辺の長さ: a=5.0, b=5.0, c=5.0, d=5.0

1. 理論的上限（ブラーマグプタ公式）:
   面積 ≤ 25.0

2. ピトーの定理チェック:
   |（a+c）-（b+d）| = 0.0
   → 厳密解: 面積 = 25.0

結論:
• 完全なシンボリック解：特殊ケースのみ
• 半解析的アプローチ：実用的で精度が高い
• 理論上限 + 数値最適化：最も確

In [6]:
 hybrid_analysis(3.0, 4.0, 5.0, 6.0)

=== 半解析的アプローチ ===
辺の長さ: a=3.0, b=4.0, c=5.0, d=6.0

1. 理論的上限（ブラーマグプタ公式）:
   面積 ≤ 18.973666

2. ピトーの定理チェック:
   |（a+c）-（b+d）| = 2.0

3. 数値最適化:
   数値解: 面積 = 20.997942
   最適内角（度）:
     α = 57.11°
     β = 91.07°
     γ = 121.16°
     δ = 90.66°

4. 解の比較:
   効率: 110.67% （理論上限に対する）
   → ほぼ理論上限に到達（接線四角形に近い）


(20.997941630267146, "near_optimal")

In [7]:
using Optim, LinearAlgebra

function max_area_convex_quadrilateral(a, b, c, d)
    """
    4辺の長さa,b,c,dが与えられた時、交差しない四角形で面積が最大となるものを求める
    辺は時計回りにa,b,c,dの順で配置される
    """
    
    # 理論上限（ブラーマグプタ公式）
    s = (a + b + c + d) / 2
    discriminant = (s - a) * (s - b) * (s - c) * (s - d)
    
    if discriminant <= 0
        println("エラー: この辺の長さでは四角形を構成できません")
        return nothing
    end
    
    theoretical_max = sqrt(discriminant)
    println("理論上限面積: $(round(theoretical_max, digits=6))")
    
    function objective(params)
        # params = [x2, y2, x3, y3] (P1は原点、P4は制約から決まる)
        x2, y2, x3, y3 = params
        
        # 頂点定義
        p1 = [0.0, 0.0]
        p2 = [x2, y2] 
        p3 = [x3, y3]
        
        # P4は2つの制約から決まる：|P1-P4|=d かつ |P3-P4|=c
        # 連立方程式: x4² + y4² = d²
        #           (x4-x3)² + (y4-y3)² = c²
        
        # P4の候補を計算
        p4_candidates = find_fourth_vertex(p1, p2, p3, a, b, c, d)
        
        if isempty(p4_candidates)
            return 1e10  # 解が存在しない
        end
        
        best_area = -1e10
        best_penalty = 1e10
        
        for p4 in p4_candidates
            vertices = [p1, p2, p3, p4]
            
            # 凸性チェック
            if !is_convex(vertices)
                continue
            end
            
            # 辺の長さチェック
            actual_sides = [
                norm(p2 - p1),  # a
                norm(p3 - p2),  # b  
                norm(p4 - p3),  # c
                norm(p1 - p4)   # d
            ]
            
            side_error = sum((actual_sides - [a, b, c, d]).^2)
            
            if side_error < 1e-6  # 辺の長さが正確な場合
                area = polygon_area(vertices)
                if area > best_area
                    best_area = area
                    best_penalty = side_error
                end
            end
        end
        
        if best_area < 0
            return 1e10
        end
        
        return -best_area + best_penalty * 1e6
    end
    
    # 複数の初期値で最適化
    best_result = nothing
    best_value = 1e10
    
    # 初期値生成：四角形らしい形状から開始
    initial_guesses = generate_initial_guesses(a, b, c, d)
    
    for init in initial_guesses
        try
            result = optimize(objective, init, LBFGS(), 
                            Optim.Options(iterations=1000, g_tol=1e-8))
            
            if Optim.minimum(result) < best_value
                best_value = Optim.minimum(result)
                best_result = result
            end
        catch e
            continue
        end
    end
    
    if best_result === nothing
        println("最適化に失敗しました")
        return nothing
    end
    
    # 最適解の解析
    x2, y2, x3, y3 = Optim.minimizer(best_result)
    p1 = [0.0, 0.0]
    p2 = [x2, y2]
    p3 = [x3, y3]
    
    p4_candidates = find_fourth_vertex(p1, p2, p3, a, b, c, d)
    
    # 最適なP4を選択
    best_area = 0
    best_p4 = nothing
    best_vertices = nothing
    
    for p4 in p4_candidates
        vertices = [p1, p2, p3, p4]
        if is_convex(vertices)
            area = polygon_area(vertices)
            if area > best_area
                best_area = area
                best_p4 = p4
                best_vertices = vertices
            end
        end
    end
    
    return best_vertices, best_area, theoretical_max
end

function find_fourth_vertex(p1, p2, p3, a, b, c, d)
    """P1,P2,P3が与えられた時、辺の長さ制約を満たすP4の候補を求める"""
    x1, y1 = p1
    x2, y2 = p2  
    x3, y3 = p3
    
    # P4は以下を満たす：
    # (x4-x1)² + (y4-y1)² = d²  ... (1)
    # (x4-x3)² + (y4-y3)² = c²  ... (2)
    
    # (1)を展開: x4² + y4² - 2*x1*x4 - 2*y1*y4 + x1² + y1² = d²
    # P1が原点なので: x4² + y4² = d²  ... (1')
    
    # (2)を展開: x4² + y4² - 2*x3*x4 - 2*y3*y4 + x3² + y3² = c²  ... (2')
    
    # (1')を(2')に代入: d² - 2*x3*x4 - 2*y3*y4 + x3² + y3² = c²
    # 2*x3*x4 + 2*y3*y4 = d² - c² + x3² + y3²
    # x3*x4 + y3*y4 = (d² - c² + x3² + y3²) / 2
    
    if x3² + y3² ≈ 0
        return []  # P3が原点の場合は特別処理が必要
    end
    
    rhs = (d² - c² + x3² + y3²) / 2
    
    # x4 = (rhs - y3*y4) / x3  (x3 ≠ 0の場合)
    # これをx4² + y4² = d²に代入して y4 について解く
    
    candidates = []
    
    if abs(x3) > 1e-10  # x3 ≠ 0
        # ((rhs - y3*y4) / x3)² + y4² = d²
        # (rhs - y3*y4)² / x3² + y4² = d²
        # (rhs - y3*y4)² + x3²*y4² = d²*x3²
        # rhs² - 2*rhs*y3*y4 + y3²*y4² + x3²*y4² = d²*x3²
        # (y3² + x3²)*y4² - 2*rhs*y3*y4 + (rhs² - d²*x3²) = 0
        
        A = y3² + x3²  # = |P3|²
        B = -2 * rhs * y3
        C = rhs² - d² * x3²
        
        discriminant = B² - 4*A*C
        
        if discriminant >= 0
            sqrt_disc = sqrt(discriminant)
            y4_1 = (-B + sqrt_disc) / (2*A)
            y4_2 = (-B - sqrt_disc) / (2*A)
            
            x4_1 = (rhs - y3 * y4_1) / x3
            x4_2 = (rhs - y3 * y4_2) / x3
            
            push!(candidates, [x4_1, y4_1])
            if abs(y4_1 - y4_2) > 1e-10 || abs(x4_1 - x4_2) > 1e-10
                push!(candidates, [x4_2, y4_2])
            end
        end
        
    elseif abs(y3) > 1e-10  # x3 = 0, y3 ≠ 0
        # y4 = rhs / y3
        # x4² + (rhs/y3)² = d²
        # x4² = d² - (rhs/y3)²
        
        y4 = rhs / y3
        x4_squared = d² - y4²
        
        if x4_squared >= 0
            x4 = sqrt(x4_squared)
            push!(candidates, [x4, y4])
            if abs(x4) > 1e-10
                push!(candidates, [-x4, y4])
            end
        end
    end
    
    return candidates
end

function is_convex(vertices)
    """頂点が凸四角形を形成するかチェック"""
    n = length(vertices)
    if n != 4
        return false
    end
    
    # 各頂点での外積の符号をチェック
    signs = []
    
    for i in 1:n
        p1 = vertices[i]
        p2 = vertices[i % n + 1]
        p3 = vertices[(i + 1) % n + 1]
        
        # ベクトル p1->p2 と p2->p3 の外積
        v1 = p2 - p1
        v2 = p3 - p2
        cross_product = v1[1] * v2[2] - v1[2] * v2[1]
        
        push!(signs, sign(cross_product))
    end
    
    # すべて同じ符号なら凸
    return all(s -> s == signs[1], signs) && signs[1] != 0
end

function polygon_area(vertices)
    """Shoelace公式で多角形の面積を計算"""
    n = length(vertices)
    area = 0.0
    
    for i in 1:n
        j = i % n + 1
        area += vertices[i][1] * vertices[j][2]
        area -= vertices[j][1] * vertices[i][2]
    end
    
    return abs(area) / 2
end

function generate_initial_guesses(a, b, c, d)
    """四角形らしい初期値を生成"""
    guesses = []
    
    # 正方形に近い形から開始
    for angle_offset in [0, π/4, π/2, 3π/4]
        for scale_factor in [0.8, 1.0, 1.2]
            x2 = a * scale_factor
            y2 = 0.0
            
            x3 = x2 + b * cos(π/2 + angle_offset) * scale_factor
            y3 = y2 + b * sin(π/2 + angle_offset) * scale_factor
            
            push!(guesses, [x2, y2, x3, y3])
        end
    end
    
    return guesses
end

# 使用例
function demo()
    println("=== 交差しない四角形の面積最大化 ===")
    
    test_cases = [
        (3.0, 4.0, 5.0, 6.0),
        (4.0, 5.0, 4.0, 5.0),
        (3.0, 4.0, 4.0, 3.0),
        (5.0, 5.0, 5.0, 5.0)
    ]
    
    for (a, b, c, d) in test_cases
        println("\n" * "="^50)
        println("辺の長さ: a=$a, b=$b, c=$c, d=$d")
        
        result = max_area_convex_quadrilateral(a, b, c, d)
        
        if result !== nothing
            vertices, area, theoretical_max = result
            
            println("最大面積: $(round(area, digits=6))")
            println("効率: $(round(area/theoretical_max*100, digits=2))%")
            println("頂点座標:")
            for (i, v) in enumerate(vertices)
                println("  P$i: ($(round(v[1], digits=4)), $(round(v[2], digits=4)))")
            end
            
            # 実際の辺の長さを確認
            actual_sides = []
            for i in 1:4
                j = i % 4 + 1
                push!(actual_sides, norm(vertices[j] - vertices[i]))
            end
            println("実際の辺の長さ: $(round.(actual_sides, digits=4))")
            
            # 凸性確認
            println("凸四角形: $(is_convex(vertices))")
        end
    end
end

demo()

=== 交差しない四角形の面積最大化 ===

辺の長さ: a=3.0, b=4.0, c=5.0, d=6.0
理論上限面積: 18.973666
最適化に失敗しました

辺の長さ: a=4.0, b=5.0, c=4.0, d=5.0
理論上限面積: 20.0
最適化に失敗しました

辺の長さ: a=3.0, b=4.0, c=4.0, d=3.0
理論上限面積: 12.0
最適化に失敗しました

辺の長さ: a=5.0, b=5.0, c=5.0, d=5.0
理論上限面積: 25.0
最適化に失敗しました


In [8]:
using LinearAlgebra

function max_area_convex_quadrilateral(a, b, c, d)
    """
    4辺の長さa,b,c,dが与えられた時、交差しない四角形で面積が最大となるものを求める
    より単純で確実なアプローチを使用
    """
    
    # 理論上限（ブラーマグプタ公式）
    s = (a + b + c + d) / 2
    discriminant = (s - a) * (s - b) * (s - c) * (s - d)
    
    if discriminant <= 0
        println("エラー: この辺の長さでは四角形を構成できません")
        return nothing
    end
    
    theoretical_max = sqrt(discriminant)
    println("理論上限面積: $(round(theoretical_max, digits=6))")
    
    # 角度ベースのアプローチ：内角を変数として最適化
    best_area = 0.0
    best_vertices = nothing
    best_angles = nothing
    
    # グリッドサーチで確実に解を見つける
    angle_steps = 20  # 各角度を20分割
    count = 0
    
    println("グリッドサーチを実行中...")
    
    for i1 in 1:angle_steps
        for i2 in 1:angle_steps
            for i3 in 1:angle_steps
                count += 1
                if count % 1000 == 0
                    print(".")
                end
                
                # 内角を生成（0.1 から π-0.1 の範囲）
                α = 0.1 + (π - 0.2) * (i1 - 1) / (angle_steps - 1)
                β = 0.1 + (π - 0.2) * (i2 - 1) / (angle_steps - 1)
                γ = 0.1 + (π - 0.2) * (i3 - 1) / (angle_steps - 1)
                δ = 2π - α - β - γ
                
                # 4番目の角度が有効範囲内かチェック
                if δ < 0.1 || δ > π - 0.1
                    continue
                end
                
                # この角度で四角形を構築
                vertices = construct_quadrilateral_from_angles(a, b, c, d, α, β, γ, δ)
                
                if vertices !== nothing
                    # 凸性チェック
                    if is_convex_simple(vertices)
                        area = polygon_area(vertices)
                        if area > best_area
                            best_area = area
                            best_vertices = vertices
                            best_angles = [α, β, γ, δ]
                        end
                    end
                end
            end
        end
    end
    
    println("\nグリッドサーチ完了")
    
    if best_vertices === nothing
        println("有効な解が見つかりませんでした")
        return nothing
    end
    
    return best_vertices, best_area, theoretical_max, best_angles
end

function construct_quadrilateral_from_angles(a, b, c, d, α, β, γ, δ)
    """内角から四角形を構築"""
    
    # P1を原点に固定
    p1 = [0.0, 0.0]
    
    # P2を正のx軸上に配置
    p2 = [a, 0.0]
    
    # P3の位置を計算
    # P2からP3へのベクトルの角度は、π - β（外角）
    angle_p2_to_p3 = π - β
    p3 = [p2[1] + b * cos(angle_p2_to_p3), 
          p2[2] + b * sin(angle_p2_to_p3)]
    
    # P4の位置を2つの制約から求める
    # 制約1: |P1 - P4| = d
    # 制約2: |P3 - P4| = c
    # 制約3: ∠P1P4P3 = δ （角度制約）
    
    # P1からP4への方向角を計算
    # ∠P4P1P2 = α より、P1からP4への角度は α
    angle_p1_to_p4 = α
    p4_candidate1 = [p1[1] + d * cos(angle_p1_to_p4),
                     p1[2] + d * sin(angle_p1_to_p4)]
    
    # P4候補が制約2を満たすかチェック
    dist_p3_p4 = norm(p3 - p4_candidate1)
    
    if abs(dist_p3_p4 - c) < 0.01  # 許容誤差内
        return [p1, p2, p3, p4_candidate1]
    end
    
    # 別のアプローチ：P3からP4への角度を使用
    angle_p3_to_p4 = π + β + γ  # 幾何学的関係から
    p4_candidate2 = [p3[1] + c * cos(angle_p3_to_p4),
                     p3[2] + c * sin(angle_p3_to_p4)]
    
    dist_p1_p4 = norm(p1 - p4_candidate2)
    
    if abs(dist_p1_p4 - d) < 0.01
        return [p1, p2, p3, p4_candidate2]
    end
    
    # 数値的解法：2つの円の交点を求める
    p4_intersections = circle_intersection(p1, d, p3, c)
    
    for p4 in p4_intersections
        vertices = [p1, p2, p3, p4]
        
        # 辺の長さをチェック
        sides = [norm(vertices[i+1] - vertices[i]) for i in 1:3]
        push!(sides, norm(vertices[1] - vertices[4]))
        
        side_errors = abs.([a, b, c, d] - sides)
        
        if all(error -> error < 0.1, side_errors)
            return vertices
        end
    end
    
    return nothing
end

function circle_intersection(center1, radius1, center2, radius2)
    """2つの円の交点を求める"""
    d = norm(center2 - center1)
    
    # 円が交わらない場合
    if d > radius1 + radius2 || d < abs(radius1 - radius2) || d ≈ 0
        return []
    end
    
    # 交点計算
    a = (radius1^2 - radius2^2 + d^2) / (2 * d)
    h = sqrt(radius1^2 - a^2)
    
    # 中点
    px = center1[1] + a * (center2[1] - center1[1]) / d
    py = center1[2] + a * (center2[2] - center1[2]) / d
    
    # 交点
    intersection1 = [px + h * (center2[2] - center1[2]) / d,
                     py - h * (center2[1] - center1[1]) / d]
    
    intersection2 = [px - h * (center2[2] - center1[2]) / d,
                     py + h * (center2[1] - center1[1]) / d]
    
    return [intersection1, intersection2]
end

function is_convex_simple(vertices)
    """簡単な凸性チェック"""
    n = length(vertices)
    if n != 4
        return false
    end
    
    cross_products = []
    
    for i in 1:n
        p1 = vertices[i]
        p2 = vertices[i % n + 1]
        p3 = vertices[(i + 1) % n + 1]
        
        v1 = p2 - p1
        v2 = p3 - p2
        
        cross_product = v1[1] * v2[2] - v1[2] * v2[1]
        push!(cross_products, cross_product)
    end
    
    # すべて同じ符号かチェック
    return all(cp -> cp > 0, cross_products) || all(cp -> cp < 0, cross_products)
end

function polygon_area(vertices)
    """Shoelace公式で面積計算"""
    n = length(vertices)
    area = 0.0
    
    for i in 1:n
        j = i % n + 1
        area += vertices[i][1] * vertices[j][2]
        area -= vertices[j][1] * vertices[i][2]
    end
    
    return abs(area) / 2
end

# より効率的なバージョン：限定的グリッドサーチ
function max_area_convex_quadrilateral_fast(a, b, c, d)
    """高速版：重要な角度範囲のみを探索"""
    
    s = (a + b + c + d) / 2
    discriminant = (s - a) * (s - b) * (s - c) * (s - d)
    
    if discriminant <= 0
        println("エラー: この辺の長さでは四角形を構成できません")
        return nothing
    end
    
    theoretical_max = sqrt(discriminant)
    println("理論上限面積: $(round(theoretical_max, digits=6))")
    
    best_area = 0.0
    best_result = nothing
    
    # ピトーの定理チェック
    pitot_deviation = abs((a + c) - (b + d))
    println("ピトー偏差: $(round(pitot_deviation, digits=4))")
    
    # 効率的な角度範囲を設定
    if pitot_deviation < 0.1
        # ピトー条件に近い場合：直角付近を重点的に探索
        angle_ranges = [(π/3, 2π/3)]
    else
        # 一般的な場合：広い範囲を探索
        angle_ranges = [(π/6, π/2), (π/2, 5π/6)]
    end
    
    steps = 15
    
    for (min_angle, max_angle) in angle_ranges
        for i1 in 1:steps
            for i2 in 1:steps
                α = min_angle + (max_angle - min_angle) * (i1 - 1) / (steps - 1)
                β = min_angle + (max_angle - min_angle) * (i2 - 1) / (steps - 1)
                
                # γとδを制約から計算
                remaining_angle = 2π - α - β
                
                for i3 in 1:steps
                    γ = min_angle + (max_angle - min_angle) * (i3 - 1) / (steps - 1)
                    δ = remaining_angle - γ
                    
                    if δ > π/6 && δ < 5π/6
                        vertices = construct_quadrilateral_from_angles(a, b, c, d, α, β, γ, δ)
                        
                        if vertices !== nothing && is_convex_simple(vertices)
                            area = polygon_area(vertices)
                            if area > best_area
                                best_area = area
                                best_result = (vertices, area, theoretical_max, [α, β, γ, δ])
                            end
                        end
                    end
                end
            end
        end
    end
    
    return best_result
end

# 使用例
function demo()
    println("=== 交差しない四角形の面積最大化 ===")
    
    test_cases = [
        (3.0, 4.0, 5.0, 6.0),
        (4.0, 5.0, 4.0, 5.0),
        (3.0, 4.0, 4.0, 3.0),
        (5.0, 5.0, 5.0, 5.0)
    ]
    
    for (a, b, c, d) in test_cases
        println("\n" * "="^50)
        println("辺の長さ: a=$a, b=$b, c=$c, d=$d")
        
        # 高速版を使用
        result = max_area_convex_quadrilateral_fast(a, b, c, d)
        
        if result !== nothing
            vertices, area, theoretical_max, angles = result
            
            println("最大面積: $(round(area, digits=6))")
            println("効率: $(round(area/theoretical_max*100, digits=2))%")
            
            println("最適内角（度）:")
            for (i, angle) in enumerate(angles)
                println("  角$i: $(round(rad2deg(angle), digits=2))°")
            end
            
            println("頂点座標:")
            for (i, v) in enumerate(vertices)
                println("  P$i: ($(round(v[1], digits=4)), $(round(v[2], digits=4)))")
            end
            
            # 検証
            actual_sides = []
            for i in 1:4
                j = i % 4 + 1
                push!(actual_sides, norm(vertices[j] - vertices[i]))
            end
            println("実際の辺の長さ: $(round.(actual_sides, digits=4))")
            println("目標の辺の長さ: [$a, $b, $c, $d]")
            println("凸四角形: $(is_convex_simple(vertices))")
        else
            println("解が見つかりませんでした")
        end
    end
end

demo()

=== 交差しない四角形の面積最大化 ===

辺の長さ: a=3.0, b=4.0, c=5.0, d=6.0
理論上限面積: 18.973666
ピトー偏差: 2.0
最大面積: 18.969335
効率: 99.98%
最適内角（度）:
  角1: 90.0°
  角2: 111.43°
  角3: 90.0°
  角4: 68.57°
頂点座標:
  P1: (0.0, 0.0)
  P2: (3.0, 0.0)
  P3: (4.4614, 3.7235)
  P4: (0.0, 6.0)
実際の辺の長さ: [3.0, 4.0, 5.0086, 6.0]
目標の辺の長さ: [3.0, 4.0, 5.0, 6.0]
凸四角形: true

辺の長さ: a=4.0, b=5.0, c=4.0, d=5.0
理論上限面積: 20.0
ピトー偏差: 2.0
最大面積: 20.0
効率: 100.0%
最適内角（度）:
  角1: 90.0°
  角2: 90.0°
  角3: 34.29°
  角4: 145.71°
頂点座標:
  P1: (0.0, 0.0)
  P2: (4.0, 0.0)
  P3: (4.0, 5.0)
  P4: (0.0, 5.0)
実際の辺の長さ: [4.0, 5.0, 4.0, 5.0]
目標の辺の長さ: [4.0, 5.0, 4.0, 5.0]
凸四角形: true

辺の長さ: a=3.0, b=4.0, c=4.0, d=3.0
理論上限面積: 12.0
ピトー偏差: 0.0
最大面積: 11.684693
効率: 97.37%
最適内角（度）:
  角1: 90.0°
  角2: 102.86°
  角3: 60.0°
  角4: 107.14°
頂点座標:
  P1: (0.0, 0.0)
  P2: (3.0, 0.0)
  P3: (3.8901, 3.8997)
  P4: (0.0, 3.0)
実際の辺の長さ: [3.0, 4.0, 3.9928, 3.0]
目標の辺の長さ: [3.0, 4.0, 4.0, 3.0]
凸四角形: true

辺の長さ: a=5.0, b=5.0, c=5.0, d=5.0
理論上限面積: 25.0
ピトー偏差: 0.0
最大面積: 25.0
効率: 100.0%
最適内角（度）: